In [1]:
import pybedtools as pbt
import pysam
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as ss

In [2]:
import splanl.junction_scorer as jn
import splanl.merge_bcs as mbcs
import splanl.coords as cds
import splanl.plots as sp
import splanl.score_motifs as sm
import splanl.inspect_variants as iv
import splanl.post_processing as pp
import splanl.custom_splai_scores as css

Using TensorFlow backend.


In [3]:
chr11 = '/nfs/kitzman3/lab_common/refs/human/ucsc_hg19/bychrom/chr11.fa'
chr11_seq = pp.get_refseq( chr11 )[ 0 ]

In [4]:
data_in = '/nfs/kitzman2/smithcat/proj/wt1_2022/ex9_data/'

In [5]:
byvartbl_wide = pd.read_table( data_in + 'wt1_ex9_by_var_effects_snvs_wide-2022-0727.txt' )

In [6]:
lit = pd.read_table( data_in + 'wt1_ex9_variant_lit.2022-0721.txt' )

In [7]:
lit

,hg19_pos,cDNA_pos,cDNA ref,cDNA alt,In ClinVar?,ClinVar interp,Refeence from ClinVar,source where variant were reported,Interpretation_ACMG_Varsome,Human disease reported in HGMD,In vitro evidence from other literatures,Reference from HGMD,Note
0,32413697,c.1355-87,G,A,Yes,likely benign,GeneDx Variant Classification Process,ClinVar,Benign,NaN,NaN,NaN,NaN
1,32413620,c.1355-10,G,C,Yes,likely benign,Invitae Variant Classification Sherloc,ClinVar,Likely benign,NaN,NaN,NaN,NaN
2,32413620,c.1355-10,G,A,No,NaN,NaN,HGMD,VUS,Breast cancer?,NaN,"Park (2018) BMC Cancer 18,",NaN
3,32413618,c.1355-8,C,T,Yes,likely benign,Invitae Variant Classification Sherloc,ClinVar,VUS,NaN,NaN,NaN,NaN
4,32413617,c.1355-7,C,T,Yes,likely benign,Invitae Variant Classification Sherloc,ClinVar,Likely benign,NaN,NaN,NaN,NaN
5,32413611,c.1355-1,G,A,No,NaN,NaN,HGMD,Pathogenic,FSGS,NaN,"Kanemoto (2007) Pediatr Nephrol 22, 454","Although not reported in ClinVar, it was repor..."
6,32413517,c.1447+1,G,A,No,NaN,NaN,HGMD,Pathogenic,Frasier syndrome,NaN,"Miyoshi (2006) Endocr J 53, 371","Although not reported in ClinVar, it was repor..."
7,32413517,c.1447+1,G,C,No,NaN,NaN,HGMD,Pathogenic,FSGS,NaN,"Gast (2016) Nephrol Dial Transplant 31, 961","Although not reported in ClinVar, it was repor..."
8,32413516,c.1447+2,T,C,Yes,likely pathogenic,"Kikuchi (1998) J Med Genet 35, 45",ClinVar & HGMD,Pathogenic,Frasier syndrome,Loss of +KTS isoform in minigene assay and in ...,"Kikuchi (1998) J Med Genet 35, 45",NaN
9,32413515,c.1447+3,G,A,Yes,VUS,Invitae Variant Classification Sherloc,ClinVar,Pathogenic,NaN,NaN,NaN,NaN


In [8]:
lit = lit.rename( columns = { 'cDNA ref': 'ref_c',
                              'cDNA alt': 'alt_c',
                              'cDNA_pos': 'hgvs_pos' } )

In [9]:
lit[ 'ref' ] = [ css.rev_complement( r ) for r in lit.ref_c ]

In [10]:
lit[ 'alt' ] = [ css.rev_complement( a ) for a in lit.alt_c ]

In [11]:
#ran this as a left merge bc one of the lit variants is too far into US intron (c.1355-87)
idx_cols = [ 'hgvs_pos', 'hg19_pos', 'ref', 'alt', 'ref_c', 'alt_c' ]

byvartbl_wide = byvartbl_wide.set_index( idx_cols ).merge( lit.set_index( idx_cols ),
                                                           how = 'left',
                                                           left_index = True,
                                                           right_index = True ).reset_index()

In [12]:
byvartbl_wide[ 'clinvar' ] = byvartbl_wide[ 'ClinVar interp' ].notnull()

In [13]:
byvartbl_wide[ 'acmg' ] = byvartbl_wide[ 'Interpretation_ACMG_Varsome' ].notnull()

In [14]:
( ( byvartbl_wide.clinvar ) | ( byvartbl_wide.acmg ) ).sum()

31

In [15]:
byvartbl_wide[ 'lit_interp' ] = [ 'Likely pathogenic/pathogenic (ClinVar)' if i == 'pathogenic' or i == 'likely pathogenic' \
                                  else 'VUS/Conflicting' if i == 'VUS' or i == 'conflicting interpretations' \
                                  else 'Likely benign/benign' if i == 'benign' or i == 'likely benign' \
                                  else '' for i in byvartbl_wide[ 'ClinVar interp' ] ]

In [16]:
byvartbl_wide.loc[ ( byvartbl_wide.Interpretation_ACMG_Varsome == 'Pathogenic' ) & ( byvartbl_wide.lit_interp == '' ), 'lit_interp' ] = 'Likely pathogenic/pathogenic (Literature)'

In [17]:
byvartbl_wide[ 'KTS_+_ratio_log2' ] = np.log2( byvartbl_wide[ 'wmean_KTS+'] / byvartbl_wide[ 'wmean_KTS-' ] )

In [18]:
byvartbl_wide.to_csv( data_in + 'wt1_ex9_by_var_effects_snvs_wide-2022-0727b.txt',
                      sep = '\t',
                      index = False )